# 날씨 데이터 수집 및 전처리

## 기상 데이터 전처리 개요

[목표]
- 분석에 있어서 가장 중요한 데이터인 날씨 데이터를 수집하고, 일정한 형식에 맞게 전처리를 진행한다.

[사용 데이터]
- 기상청 기상자료개방포털 단기예보 (원본사이트 : https://data.kma.go.kr/data/rmt/rmtList.do?code=420&pgmNo=572)

[기준]
- 기준 시간 : 협정세계시, 즉 영국 그리니치 천문대 기준 시간이기에, 한국 시간으로 하기 위해서는 9시간을 더해줘야 함. 그리고 거기에 추가되는 시간까지 더해줘야 진짜 한국시간으로 표시됨.
- utc기준 200, +7로 기준 잡음 즉, 오후 6시를 기준으로 잡게됨. (모든 데이터에 다 있고, 공통된지표가 있어야 해서 이를 선택함.)

[과정]
- 2019년 1월1일부터 2022년 5월 31일까지 총 1247개가 있어야 함

[변수설명]
- df1, df2, df3, df4 -> 각 데이터(2019.01.01~2022.05.31)들을 가져온 데이터프레임
- 도봉구강수
- 도봉구강설
- 도봉구최고기온
- 도봉구최저기온
- 도봉구습도
- 도봉구강수형태
- 도봉구강수확률

In [9]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [10]:
# 2019-01-01에서 2022-05-31 사이 모든 날짜를 크롤링하는 함수 만들어서 가져오기
from datetime import datetime, timedelta
# 해당 범위 내의 날짜들을 리스트로 지정해주는 함수
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates
# 2021-01-01 ~ 2021-12-31 사이의 날짜를 리스트로 지정
date_list = date_range("2019-01-01", "2022-05-31")

In [11]:
# 하나의 데이터프레임으로 합치는 함수
# 최저기온은 따로 처리를 해야함 -> 일단 다 0으로 넣어놓기
def concat_dfs(강수량_df, 강설량_df, 최고기온_df, 습도_df, 강수형태_df, 강수확률_df):
    # 인덱스 맞추기
    강수량_df.reset_index(drop=True, inplace=True)
    강설량_df.reset_index(drop=True, inplace=True)
    최고기온_df.reset_index(drop=True, inplace=True)
    습도_df.reset_index(drop=True, inplace=True)
    강수형태_df.reset_index(drop=True, inplace=True)
    강수확률_df.reset_index(drop=True, inplace=True)

    
    최저기온_list = []
    for i in range(1247):
        최저기온_list.append(0)
    
    # 하나의 데이터 프레임으로
    concated_df = pd.DataFrame()
    concated_df['Date'] = date_list
    concated_df['강수량'] = 강수량_df.iloc[:, -1]
    concated_df['강설량'] = 강설량_df.iloc[:, -1]
    concated_df['최고기온'] = 최고기온_df.iloc[:, -1]
    concated_df['최저기온'] = 최저기온_list
    concated_df['습도'] = 습도_df.iloc[:, -1]
    concated_df['강수형태'] = 강수형태_df.iloc[:, -1]
    concated_df['강수확률'] = 강수확률_df.iloc[:, -1]

    return concated_df
p = [(1+exp(b0+b1x))/(1+exp(b0+b1x))= 1/(1+exp(-b0-b1x))]
# sigmoid 함수를 사용해서 데이터 분석을 함 

log(p/1-p) = b0+sum(bixi)

## 강설량 - 도봉구
- 도봉구의 경우에는 도봉제1동의 데이터를 대표로 설정하였음

In [12]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강설량/도봉제1동_6시간적설_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강설량/도봉제1동_6시간적설_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강설량/도봉제1동_6시간적설_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강설량/도봉제1동_1시간적설_20210701_20220531.csv')

In [1]:
df1.info(), df2.info(), df3.info(), df4.info()

NameError: name 'df1' is not defined

In [14]:
# df1.columns = ['예측날짜','예측시간','추가시간','강수량']

In [15]:
df1[(df1['hour']==200)&(df1['forecast']==7)]

,format: day,hour,forecast,value location:61_129 Start : 20190101
0,1,200.0,7.0,0.0
76,2,200.0,7.0,0.0
152,3,200.0,7.0,0.0
228,4,200.0,7.0,0.0
304,5,200.0,7.0,0.0
...,...,...,...,...
27371,27,200.0,7.0,0.0
27447,28,200.0,7.0,0.0
27523,29,200.0,7.0,0.0
27599,30,200.0,7.0,0.0


In [16]:
df2[(df2['hour']==200)&(df2['forecast']==7)]

,format: day,hour,forecast,value location:61_129 Start : 20200101
0,1,200.0,7.0,0.0
76,2,200.0,7.0,0.0
152,3,200.0,7.0,0.0
228,4,200.0,7.0,0.0
304,5,200.0,7.0,0.0
...,...,...,...,...
27447,27,200.0,7.0,0.0
27523,28,200.0,7.0,0.0
27599,29,200.0,7.0,1.0
27675,30,200.0,7.0,0.0


In [17]:
df3[(df3['hour']==200)&(df3['forecast']==7)]

,format: day,hour,forecast,value location:61_129 Start : 20210101
0,1,200.0,7.0,0.0
76,2,200.0,7.0,0.0
152,3,200.0,7.0,0.0
228,4,200.0,7.0,0.0
304,5,200.0,7.0,0.0
...,...,...,...,...
13381,26,200.0,7.0,0.0
13457,27,200.0,7.0,0.0
13533,28,200.0,7.0,0.0
13609,29,200.0,7.0,0.0


In [18]:
df4[(df4['hour']==200)&(df4['forecast']==7)]

,format: day,hour,forecast,value location:61_129 Start : 20210701
1,1,200.0,7.0,0.0
510,2,200.0,7.0,0.0
1019,3,200.0,7.0,0.0
1528,4,200.0,7.0,0.0
2037,5,200.0,7.0,0.0
...,...,...,...,...
167971,27,200.0,7.0,0.0
168480,28,200.0,7.0,0.0
168989,29,200.0,7.0,0.0
169498,30,200.0,7.0,0.0


- 2019년 1월1일부터 2022년 5월 31일까지 총 1247개가 있어야 함.

In [19]:
a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

In [20]:
a.columns = ['일','측정시간','이후시간예측','강설량']
b.columns = ['일','측정시간','이후시간예측','강설량']
c.columns = ['일','측정시간','이후시간예측','강설량']
d.columns = ['일','측정시간','이후시간예측','강설량']

In [21]:
도봉구강설량 = pd.concat([a,b,c,d])

In [22]:
도봉구강설량

,일,측정시간,이후시간예측,강설량
0,1,200.0,7.0,0.0
76,2,200.0,7.0,0.0
152,3,200.0,7.0,0.0
228,4,200.0,7.0,0.0
304,5,200.0,7.0,0.0
...,...,...,...,...
167971,27,200.0,7.0,0.0
168480,28,200.0,7.0,0.0
168989,29,200.0,7.0,0.0
169498,30,200.0,7.0,0.0


- 강설량은 적절하게 완료된 것을 확인할 수 있음.

## 강수량 - 도봉구

In [23]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수량/도봉제1동_6시간강수량_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수량/도봉제1동_6시간강수량_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수량/도봉제1동_6시간강수량_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수량/도봉제1동_1시간강수량_20210701_20220531.csv')

In [24]:
df1.info(), df2.info(), df3.info(), df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27751 entries, 0 to 27750
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0    format: day                              27751 non-null  object 
 1   hour                                      27740 non-null  float64
 2   forecast                                  27740 non-null  float64
 3   value  location:61_129 Start : 20190101   27740 non-null  float64
dtypes: float64(3), object(1)
memory usage: 867.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27827 entries, 0 to 27826
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0    format: day                              27827 non-null  object 
 1   hour                                      27816 non-null  float64
 2   forecast       

(None, None, None, None)

In [25]:
a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

In [26]:
a.columns = ['일','측정시간','이후시간예측','강수량']
b.columns = ['일','측정시간','이후시간예측','강수량']
c.columns = ['일','측정시간','이후시간예측','강수량']
d.columns = ['일','측정시간','이후시간예측','강수량']

In [27]:
도봉구강수량 = pd.concat([a,b,c,d])
도봉구강수량

,일,측정시간,이후시간예측,강수량
0,1,200.0,7.0,0.0
76,2,200.0,7.0,0.0
152,3,200.0,7.0,0.0
228,4,200.0,7.0,0.0
304,5,200.0,7.0,0.0
...,...,...,...,...
167971,27,200.0,7.0,0.0
168480,28,200.0,7.0,0.0
168989,29,200.0,7.0,0.0
169498,30,200.0,7.0,0.0


## 강수형태 - 종로구

In [28]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수형태/도봉제1동_강수형태_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수형태/도봉제1동_강수형태_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수형태/도봉제1동_강수형태_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수형태/도봉제1동_강수형태_20210701_20220531.csv')

In [29]:
df1.info(), df2.info(), df3.info(), df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54031 entries, 0 to 54030
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0    format: day                              54031 non-null  object 
 1   hour                                      54020 non-null  float64
 2   forecast                                  54020 non-null  float64
 3   value  location:61_129 Start : 20190101   54020 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54179 entries, 0 to 54178
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0    format: day                              54179 non-null  object 
 1   hour                                      54168 non-null  float64
 2   forecast         

(None, None, None, None)

In [30]:
a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

In [31]:
a.columns = ['일','측정시간','이후시간예측','강수형태']
b.columns = ['일','측정시간','이후시간예측','강수형태']
c.columns = ['일','측정시간','이후시간예측','강수형태']
d.columns = ['일','측정시간','이후시간예측','강수형태']

In [32]:
도봉구강수형태 = pd.concat([a,b,c,d])
도봉구강수형태

,일,측정시간,이후시간예측,강수형태
1,1,200.0,7.0,0.0
149,2,200.0,7.0,0.0
297,3,200.0,7.0,0.0
445,4,200.0,7.0,0.0
593,5,200.0,7.0,0.0
...,...,...,...,...
167971,27,200.0,7.0,0.0
168480,28,200.0,7.0,0.0
168989,29,200.0,7.0,0.0
169498,30,200.0,7.0,0.0


## 강수확률 - 도봉구

In [33]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수확률/도봉제1동_강수확률_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수확률/도봉제1동_강수확률_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수확률/도봉제1동_강수확률_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/강수확률/도봉제1동_강수확률_20210701_20220531.csv')

In [34]:
df1.info(), df2.info(), df3.info(), df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54031 entries, 0 to 54030
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0    format: day                              54031 non-null  object 
 1   hour                                      54020 non-null  float64
 2   forecast                                  54020 non-null  float64
 3   value  location:61_129 Start : 20190101   54020 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54179 entries, 0 to 54178
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0    format: day                              54179 non-null  object 
 1   hour                                      54168 non-null  float64
 2   forecast         

(None, None, None, None)

In [35]:
a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

In [36]:
a.columns = ['일','측정시간','이후시간예측','강수확률']
b.columns = ['일','측정시간','이후시간예측','강수확률']
c.columns = ['일','측정시간','이후시간예측','강수확률']
d.columns = ['일','측정시간','이후시간예측','강수확률']

In [37]:
도봉구강수확률 = pd.concat([a,b,c,d])
도봉구강수확률

,일,측정시간,이후시간예측,강수확률
1,1,200.0,7.0,0.0
149,2,200.0,7.0,0.0
297,3,200.0,7.0,0.0
445,4,200.0,7.0,20.0
593,5,200.0,7.0,10.0
...,...,...,...,...
167971,27,200.0,7.0,0.0
168480,28,200.0,7.0,0.0
168989,29,200.0,7.0,0.0
169498,30,200.0,7.0,30.0


## 습도 - 도봉구

In [38]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/습도/도봉제1동_습도_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/습도/도봉제1동_습도_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/습도/도봉제1동_습도_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/습도/도봉제1동_습도_20210701_20220531.csv')

df1.info(), df2.info(), df3.info(), df4.info()

a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','습도']
b.columns = ['일','측정시간','이후시간예측','습도']
c.columns = ['일','측정시간','이후시간예측','습도']
d.columns = ['일','측정시간','이후시간예측','습도']

도봉구습도 = pd.concat([a,b,c,d])
도봉구습도

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54031 entries, 0 to 54030
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0    format: day                              54031 non-null  object 
 1   hour                                      54020 non-null  float64
 2   forecast                                  54020 non-null  float64
 3   value  location:61_129 Start : 20190101   54020 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54179 entries, 0 to 54178
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0    format: day                              54179 non-null  object 
 1   hour                                      54168 non-null  float64
 2   forecast         

,일,측정시간,이후시간예측,습도
1,1,200.0,7.0,40.0
149,2,200.0,7.0,35.0
297,3,200.0,7.0,50.0
445,4,200.0,7.0,70.0
593,5,200.0,7.0,30.0
...,...,...,...,...
167971,27,200.0,7.0,50.0
168480,28,200.0,7.0,50.0
168989,29,200.0,7.0,55.0
169498,30,200.0,7.0,75.0


In [39]:
a.shape, b.shape, c.shape, d.shape

((365, 4), (366, 4), (181, 4), (335, 4))

In [40]:
df2

,format: day,hour,forecast,value location:61_129 Start : 20200101
0,1,200.0,4.0,60.0
1,1,200.0,7.0,70.0
2,1,200.0,10.0,80.0
3,1,200.0,13.0,85.0
4,1,200.0,16.0,85.0
...,...,...,...,...
54174,31,2300.0,52.0,30.0
54175,31,2300.0,55.0,25.0
54176,31,2300.0,58.0,30.0
54177,31,2300.0,61.0,45.0


## 최고기온 - 도봉구

In [41]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/최고기온/도봉제1동_일최고기온_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/최고기온/도봉제1동_일최고기온_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/최고기온/도봉제1동_일최고기온_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/최고기온/도봉제1동_일최고기온_20210701_20220531.csv')

df1.info(), df2.info(), df3.info(), df4.info()

a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','최고기온']
b.columns = ['일','측정시간','이후시간예측','최고기온']
c.columns = ['일','측정시간','이후시간예측','최고기온']
d.columns = ['일','측정시간','이후시간예측','최고기온']

도봉구최고기온 = pd.concat([a,b,c,d])
도봉구최고기온

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6946 entries, 0 to 6945
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0    format: day                              6946 non-null   object 
 1   hour                                      6935 non-null   float64
 2   forecast                                  6935 non-null   float64
 3   value  location:61_129 Start : 20190101   6935 non-null   float64
dtypes: float64(3), object(1)
memory usage: 217.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965 entries, 0 to 6964
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0    format: day                              6965 non-null   object 
 1   hour                                      6954 non-null   float64
 2   forecast           

,일,측정시간,이후시간예측,최고기온
1,1,200.0,7.0,0.0
20,2,200.0,7.0,2.0
39,3,200.0,7.0,4.0
58,4,200.0,7.0,1.0
77,5,200.0,7.0,1.0
...,...,...,...,...
7921,27,200.0,7.0,28.0
7945,28,200.0,7.0,29.0
7969,29,200.0,7.0,24.0
7993,30,200.0,7.0,26.0


In [42]:
# concat_dfs(강수량_df, 강설량_df, 최고기온_df, 최저기온_list, 습도_df, 강수형태_df, 강수확률_df)
도봉구기상데이터 = concat_dfs(도봉구강수, 도봉구강설, 도봉구최고기온, 도봉구습도, 도봉구강수형태, 도봉구강수확률)
도봉구기상데이터.to_csv('도봉구기상데이터.csv')

도봉구기상데이터

NameError: name '도봉구강수' is not defined

# 노원구

## 노원구 공릉1동 - 최고기온

In [43]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/최고기온/공릉1동_일최고기온_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/최고기온/공릉1동_일최고기온_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/최고기온/공릉1동_일최고기온_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/최고기온/공릉1동_일최고기온_20210701_20220531.csv')

- 2019년 1월1일부터 2022년 5월 31일까지 총 1247개가 있어야 함.

In [44]:
a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

In [45]:
print(a.shape, b.shape, c.shape, d.shape)
a.shape[0] + b.shape[0] + c.shape[0] + d.shape[0]

(365, 4) (366, 4) (181, 4) (335, 4)


1247

In [46]:
a.columns = ['일','측정시간','이후시간예측','최고기온']
b.columns = ['일','측정시간','이후시간예측','최고기온']
c.columns = ['일','측정시간','이후시간예측','최고기온']
d.columns = ['일','측정시간','이후시간예측','최고기온']

In [47]:
노원구최고기온 = pd.concat([a,b,c,d])
노원구최고기온

,일,측정시간,이후시간예측,최고기온
1,1,200.0,7.0,0.0
20,2,200.0,7.0,2.0
39,3,200.0,7.0,4.0
58,4,200.0,7.0,1.0
77,5,200.0,7.0,2.0
...,...,...,...,...
7921,27,200.0,7.0,29.0
7945,28,200.0,7.0,29.0
7969,29,200.0,7.0,24.0
7993,30,200.0,7.0,27.0


## 노원구 공릉1동 - 강수형태

In [48]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수형태/공릉1동_강수형태_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수형태/공릉1동_강수형태_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수형태/공릉1동_강수형태_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수형태/공릉1동_강수형태_20210701_20220531.csv')

- 2019년 1월1일부터 2022년 5월 31일까지 총 1247개가 있어야 함.

In [49]:
a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

In [50]:
print(a.shape, b.shape, c.shape, d.shape)
a.shape[0] + b.shape[0] + c.shape[0] + d.shape[0]

(365, 4) (366, 4) (181, 4) (335, 4)


1247

In [51]:
a.columns = ['일','측정시간','이후시간예측','강수형태']
b.columns = ['일','측정시간','이후시간예측','강수형태']
c.columns = ['일','측정시간','이후시간예측','강수형태']
d.columns = ['일','측정시간','이후시간예측','강수형태']

In [52]:
노원구강수형태 = pd.concat([a,b,c,d])
노원구강수형태

,일,측정시간,이후시간예측,강수형태
1,1,200.0,7.0,0.0
149,2,200.0,7.0,0.0
297,3,200.0,7.0,0.0
445,4,200.0,7.0,0.0
593,5,200.0,7.0,0.0
...,...,...,...,...
167971,27,200.0,7.0,0.0
168480,28,200.0,7.0,0.0
168989,29,200.0,7.0,0.0
169498,30,200.0,7.0,0.0


## 노원구 공릉1동 - 강설량

In [53]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강설량/공릉1동_6시간적설_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강설량/공릉1동_6시간적설_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강설량/공릉1동_6시간적설_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강설량/공릉1동_1시간적설_20210701_20220531.csv')

- 2019년 1월1일부터 2022년 5월 31일까지 총 1247개가 있어야 함.

In [54]:
a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

In [55]:
print(a.shape, b.shape, c.shape, d.shape)
a.shape[0] + b.shape[0] + c.shape[0] + d.shape[0]

(365, 4) (366, 4) (181, 4) (335, 4)


1247

In [56]:
a.columns = ['일','측정시간','이후시간예측','강설량']
b.columns = ['일','측정시간','이후시간예측','강설량']
c.columns = ['일','측정시간','이후시간예측','강설량']
d.columns = ['일','측정시간','이후시간예측','강설량']

In [57]:
노원구강설량 = pd.concat([a,b,c,d])
노원구강설량

,일,측정시간,이후시간예측,강설량
0,1,200.0,7.0,0.0
76,2,200.0,7.0,0.0
152,3,200.0,7.0,0.0
228,4,200.0,7.0,0.0
304,5,200.0,7.0,0.0
...,...,...,...,...
167971,27,200.0,7.0,0.0
168480,28,200.0,7.0,0.0
168989,29,200.0,7.0,0.0
169498,30,200.0,7.0,0.0


## 노원구 공릉1동 - 강수량

In [58]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수량/공릉1동_6시간강수량_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수량/공릉1동_6시간강수량_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수량/공릉1동_6시간강수량_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수량/공릉1동_1시간강수량_20210701_20220531.csv')

- 2019년 1월1일부터 2022년 5월 31일까지 총 1247개가 있어야 함.

In [59]:
a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

In [60]:
print(a.shape, b.shape, c.shape, d.shape)
a.shape[0] + b.shape[0] + c.shape[0] + d.shape[0]

(365, 4) (366, 4) (181, 4) (335, 4)


1247

In [61]:
a.columns = ['일','측정시간','이후시간예측','강수량']
b.columns = ['일','측정시간','이후시간예측','강수량']
c.columns = ['일','측정시간','이후시간예측','강수량']
d.columns = ['일','측정시간','이후시간예측','강수량']

In [62]:
노원구강수량 = pd.concat([a,b,c,d])
노원구강수량

,일,측정시간,이후시간예측,강수량
0,1,200.0,7.0,0.0
76,2,200.0,7.0,0.0
152,3,200.0,7.0,0.0
228,4,200.0,7.0,0.0
304,5,200.0,7.0,0.0
...,...,...,...,...
167971,27,200.0,7.0,0.0
168480,28,200.0,7.0,0.0
168989,29,200.0,7.0,0.0
169498,30,200.0,7.0,0.0


## 노원구 공릉1동 - 습도

In [63]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/습도/공릉1동_습도_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/습도/공릉1동_습도_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/습도/공릉1동_습도_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/습도/공릉1동_습도_20210701_20220531.csv')

- 2019년 1월1일부터 2022년 5월 31일까지 총 1247개가 있어야 함.

In [64]:
a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

In [65]:
print(a.shape, b.shape, c.shape, d.shape)
a.shape[0] + b.shape[0] + c.shape[0] + d.shape[0]

(365, 4) (366, 4) (181, 4) (335, 4)


1247

In [66]:
a.columns = ['일','측정시간','이후시간예측','습도']
b.columns = ['일','측정시간','이후시간예측','습도']
c.columns = ['일','측정시간','이후시간예측','습도']
d.columns = ['일','측정시간','이후시간예측','습도']

In [67]:
노원구습도 = pd.concat([a,b,c,d])
노원구습도

,일,측정시간,이후시간예측,습도
1,1,200.0,7.0,40.0
149,2,200.0,7.0,25.0
297,3,200.0,7.0,45.0
445,4,200.0,7.0,60.0
593,5,200.0,7.0,25.0
...,...,...,...,...
167971,27,200.0,7.0,50.0
168480,28,200.0,7.0,45.0
168989,29,200.0,7.0,45.0
169498,30,200.0,7.0,70.0


# 노원구 공릉1동 - 강수확률

In [68]:
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수확률/공릉1동_강수확률_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수확률/공릉1동_강수확률_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수확률/공릉1동_강수확률_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/강수확률/공릉1동_강수확률_20210701_20220531.csv')

- 2019년 1월1일부터 2022년 5월 31일까지 총 1247개가 있어야 함.

In [69]:
a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

In [70]:
print(a.shape, b.shape, c.shape, d.shape)
a.shape[0] + b.shape[0] + c.shape[0] + d.shape[0]

(365, 4) (366, 4) (181, 4) (335, 4)


1247

In [71]:
a.columns = ['일','측정시간','이후시간예측','강수확률']
b.columns = ['일','측정시간','이후시간예측','강수확률']
c.columns = ['일','측정시간','이후시간예측','강수확률']
d.columns = ['일','측정시간','이후시간예측','강수확률']

In [72]:
노원구강수확률 = pd.concat([a,b,c,d])
노원구강수확률

,일,측정시간,이후시간예측,강수확률
1,1,200.0,7.0,0.0
149,2,200.0,7.0,0.0
297,3,200.0,7.0,0.0
445,4,200.0,7.0,20.0
593,5,200.0,7.0,10.0
...,...,...,...,...
167971,27,200.0,7.0,0.0
168480,28,200.0,7.0,0.0
168989,29,200.0,7.0,0.0
169498,30,200.0,7.0,30.0


In [73]:
# concat_dfs(강수량_df, 강설량_df, 최고기온_df, 최저기온_list, 습도_df, 강수형태_df, 강수확률_df)
노원구기상데이터 = concat_dfs(노원구강수량, 노원구강설량, 노원구최고기온, 노원구습도, 노원구강수형태, 노원구강수확률)
노원구기상데이터.to_csv('노원구기상데이터.csv')

노원구기상데이터

,Date,강수량,강설량,최고기온,최저기온,습도,강수형태,강수확률
0,2019-01-01,0.0,0.0,0.0,0,40.0,0.0,0.0
1,2019-01-02,0.0,0.0,2.0,0,25.0,0.0,0.0
2,2019-01-03,0.0,0.0,4.0,0,45.0,0.0,0.0
3,2019-01-04,0.0,0.0,1.0,0,60.0,0.0,20.0
4,2019-01-05,0.0,0.0,2.0,0,25.0,0.0,10.0
...,...,...,...,...,...,...,...,...
1242,2022-05-27,0.0,0.0,29.0,0,50.0,0.0,0.0
1243,2022-05-28,0.0,0.0,29.0,0,45.0,0.0,0.0
1244,2022-05-29,0.0,0.0,24.0,0,45.0,0.0,0.0
1245,2022-05-30,0.0,0.0,27.0,0,70.0,0.0,30.0


# 서대문구

## 서대문구 남가좌제1동

In [74]:
# 서대문구 남가좌제1동 강설량
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강설량/남가좌제1동_6시간적설_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강설량/남가좌제1동_6시간적설_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강설량/남가좌제1동_6시간적설_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강설량/남가좌제1동_1시간적설_20210701_20220531.csv')

a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','강설량']
b.columns = ['일','측정시간','이후시간예측','강설량']
c.columns = ['일','측정시간','이후시간예측','강설량']
d.columns = ['일','측정시간','이후시간예측','강설량']

print(a.shape, b.shape,c.shape, d.shape)

서대문구강설량 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [75]:
# 서대문구 남가좌제1동 강수량
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수량/남가좌제1동_6시간강수량_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수량/남가좌제1동_6시간강수량_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수량/남가좌제1동_6시간강수량_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수량/남가좌제1동_1시간강수량_20210701_20220531.csv')

a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','강수량']
b.columns = ['일','측정시간','이후시간예측','강수량']
c.columns = ['일','측정시간','이후시간예측','강수량']
d.columns = ['일','측정시간','이후시간예측','강수량']

print(a.shape, b.shape,c.shape, d.shape)

서대문구강수량 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [76]:
# 서대문구 남가좌제1동 강수형태
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수형태/남가좌제1동_강수형태_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수형태/남가좌제1동_강수형태_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수형태/남가좌제1동_강수형태_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수형태/남가좌제1동_강수형태_20210701_20220531.csv')

a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','강수형태']
b.columns = ['일','측정시간','이후시간예측','강수형태']
c.columns = ['일','측정시간','이후시간예측','강수형태']
d.columns = ['일','측정시간','이후시간예측','강수형태']

print(a.shape, b.shape,c.shape, d.shape)

서대문구강수형태 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [77]:
# 서대문구 남가좌제1동 강수확률
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수확률/남가좌제1동_강수확률_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수확률/남가좌제1동_강수확률_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수확률/남가좌제1동_강수확률_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/강수확률/남가좌제1동_강수확률_20210701_20220531.csv')

a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','강수확률']
b.columns = ['일','측정시간','이후시간예측','강수확률']
c.columns = ['일','측정시간','이후시간예측','강수확률']
d.columns = ['일','측정시간','이후시간예측','강수확률']

print(a.shape, b.shape,c.shape, d.shape)

서대문구강수확률 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [78]:
# 서대문구 남가좌제1동 최고기온
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/최고기온/남가좌제1동_일최고기온_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/최고기온/남가좌제1동_일최고기온_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/최고기온/남가좌제1동_일최고기온_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/최고기온/남가좌제1동_일최고기온_20210701_20220531.csv')

a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','최고기온']
b.columns = ['일','측정시간','이후시간예측','최고기온']
c.columns = ['일','측정시간','이후시간예측','최고기온']
d.columns = ['일','측정시간','이후시간예측','최고기온']

print(a.shape, b.shape,c.shape, d.shape)

서대문구최고기온 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [79]:
# 서대문구 남가좌제1동 습도
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/습도/남가좌제1동_습도_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/습도/남가좌제1동_습도_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/습도/남가좌제1동_습도_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/습도/남가좌제1동_습도_20210701_20220531.csv')

a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','습도']
b.columns = ['일','측정시간','이후시간예측','습도']
c.columns = ['일','측정시간','이후시간예측','습도']
d.columns = ['일','측정시간','이후시간예측','습도']

print(a.shape, b.shape,c.shape, d.shape)

서대문구습도 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [80]:
# concat_dfs(강수량_df, 강설량_df, 최고기온_df, 최저기온_list, 습도_df, 강수형태_df, 강수확률_df)
서대문구기상데이터 = concat_dfs(서대문구강수량, 서대문구강설량, 서대문구최고기온, 서대문구습도, 서대문구강수형태, 서대문구강수확률)
서대문구기상데이터.to_csv('서대문구기상데이터.csv')

서대문구기상데이터

,Date,강수량,강설량,최고기온,최저기온,습도,강수형태,강수확률
0,2019-01-01,0.0,0.0,-1.0,0,40.0,0.0,0.0
1,2019-01-02,0.0,0.0,2.0,0,30.0,0.0,0.0
2,2019-01-03,0.0,0.0,4.0,0,25.0,0.0,0.0
3,2019-01-04,0.0,0.0,1.0,0,65.0,0.0,20.0
4,2019-01-05,0.0,0.0,1.0,0,25.0,0.0,10.0
...,...,...,...,...,...,...,...,...
1242,2022-05-27,0.0,0.0,29.0,0,50.0,0.0,0.0
1243,2022-05-28,0.0,0.0,29.0,0,55.0,0.0,0.0
1244,2022-05-29,0.0,0.0,24.0,0,55.0,0.0,0.0
1245,2022-05-30,0.0,0.0,26.0,0,75.0,0.0,30.0


## 은평구 갈현제1동

In [112]:
# 은평구 갈현제1동 강설량
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강설량/갈현제1동_6시간적설_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강설량/갈현제1동_6시간적설_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강설량/갈현제1동_6시간적설_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강설량/갈현제1동_1시간적설_20210701_20220531.csv')

a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','강설량']
b.columns = ['일','측정시간','이후시간예측','강설량']
c.columns = ['일','측정시간','이후시간예측','강설량']
d.columns = ['일','측정시간','이후시간예측','강설량']

print(a.shape, b.shape,c.shape, d.shape)

은평구강설량 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [113]:
# 은평구 갈현제1동 강수량
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강설량/갈현제1동_6시간적설_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강설량/갈현제1동_6시간적설_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강설량/갈현제1동_6시간적설_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강설량/갈현제1동_1시간적설_20210701_20220531.csv')


a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','강수량']
b.columns = ['일','측정시간','이후시간예측','강수량']
c.columns = ['일','측정시간','이후시간예측','강수량']
d.columns = ['일','측정시간','이후시간예측','강수량']

print(a.shape, b.shape,c.shape, d.shape)

은평구강수량 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [114]:
# 은평구 갈현제1동 강수형태
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강수형태/갈현제1동_강수형태_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강수형태/갈현제1동_강수형태_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강수형태/갈현제1동_강수형태_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강수형태/갈현제1동_강수형태_20210701_20220531.csv')


a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','강수형태']
b.columns = ['일','측정시간','이후시간예측','강수형태']
c.columns = ['일','측정시간','이후시간예측','강수형태']
d.columns = ['일','측정시간','이후시간예측','강수형태']

print(a.shape, b.shape,c.shape, d.shape)

은평구강수형태 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [117]:
# 은평구 갈현제1동 강수확률
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강수확률/갈현제1동_강수확률_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강수확률/갈현제1동_강수확률_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강수확률/갈현제1동_강수확률_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/강수확률/갈현제1동_강수확률_20210701_20220531.csv')


a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','강수확률']
b.columns = ['일','측정시간','이후시간예측','강수확률']
c.columns = ['일','측정시간','이후시간예측','강수확률']
d.columns = ['일','측정시간','이후시간예측','강수확률']

print(a.shape, b.shape,c.shape, d.shape)

은평구강수확률 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [125]:
# 은평구 갈현제1동 최고기온
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/최고기온/갈현제1동_일최고기온_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/최고기온/갈현제1동_일최고기온_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/최고기온/갈현제1동_일최고기온_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/최고기온/갈현제2동_일최고기온_20210701_20220531.csv')


a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','최고기온']
b.columns = ['일','측정시간','이후시간예측','최고기온']
c.columns = ['일','측정시간','이후시간예측','최고기온']
d.columns = ['일','측정시간','이후시간예측','최고기온']

print(a.shape, b.shape,c.shape, d.shape)

은평구최고기온 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [104]:
# 은평구 갈현제1동 습도
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/습도/갈현제1동_습도_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/습도/갈현제1동_습도_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/습도/갈현제1동_습도_202101_202106.csv')
df4 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/습도/갈현제1동_습도_20210701_20220531.csv')


a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df3['forecast']==7)]
d = df4[(df4['hour']==200)&(df4['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','습도']
b.columns = ['일','측정시간','이후시간예측','습도']
c.columns = ['일','측정시간','이후시간예측','습도']
d.columns = ['일','측정시간','이후시간예측','습도']

print(a.shape, b.shape,c.shape, d.shape)

은평구습도 = pd.concat([a,b,c,d])

(365, 4) (366, 4) (181, 4) (335, 4)


In [126]:
# concat_dfs(강수량_df, 강설량_df, 최고기온_df, 최저기온_list, 습도_df, 강수형태_df, 강수확률_df)
은평구기상데이터 = concat_dfs(은평구강수량, 은평구강설량, 은평구최고기온, 은평구습도, 은평구강수형태, 은평구강수확률)
은평구기상데이터.to_csv('은평구기상데이터.csv')

은평구기상데이터

,Date,강수량,강설량,최고기온,최저기온,습도,강수형태,강수확률
0,2019-01-01,0.0,0.0,-1.0,0,45.0,0.0,0.0
1,2019-01-02,0.0,0.0,2.0,0,35.0,0.0,0.0
2,2019-01-03,0.0,0.0,4.0,0,50.0,0.0,0.0
3,2019-01-04,0.0,0.0,1.0,0,70.0,0.0,20.0
4,2019-01-05,0.0,0.0,1.0,0,30.0,0.0,10.0
...,...,...,...,...,...,...,...,...
1242,2022-05-27,0.0,0.0,29.0,0,55.0,0.0,0.0
1243,2022-05-28,0.0,0.0,29.0,0,55.0,0.0,0.0
1244,2022-05-29,0.0,0.0,24.0,0,55.0,0.0,0.0
1245,2022-05-30,0.0,0.0,26.0,0,80.0,0.0,30.0


# 최저기온

In [127]:
# 최저기온의 경우 4번째 데이터는 잘 안나오므로 같은 기간 2년치의 데이터 평균 내기

# 2019-01-01에서 2021-06-30 사이 모든 날짜를 크롤링하는 함수 만들어서 가져오기
from datetime import datetime, timedelta

# 해당 범위 내의 날짜들을 리스트로 지정해주는 함수
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

# 2021-01-01 ~ 2021-12-31 사이의 날짜를 리스트로 지정 
date_list2 = date_range("2019-01-01", "2021-06-30")

# 타겟이 되는 날짜 범위 2019-07-01 ~ 2021-05-31
date_list3 = date_range("2019-07-01", "2021-05-31")

In [128]:
# 2년치의 최저 기온 평균을 반환
def low_temp_list(df):
    # 타겟이 되는 날짜(2년치)
    target_df = df[df['date'].isin(date_list3)]
    
    # 월 / 일 슬라이싱하여 별도의 컬럼에 추가
    months = []
    days = []

    for i in range(target_df.shape[0]):
        m = target_df['date'].iloc[i][5:7]
        d = target_df['date'].iloc[i][8:11] 
        months.append(m)
        days.append(d)

    target_df['month'] = months
    target_df['day'] = days

    # 윤년 제거(2022년은 윤년이 아니기 때문)
    drop_inx = target_df[(target_df['month'] == '02') & (target_df['day'] == '29')].index
    target_df = target_df.drop(drop_inx)
    
    # 6월 데이터는 삭제 필요(타겟 범위에 없음)
    drop_inx2 = target_df[target_df['month'] == '06'].index
    target_df = target_df.drop(drop_inx2)
    
    # 6월 데이터는 삭제 필요(타겟 범위에 없음)
    drop_inx2 = target_df[target_df['month'] == '06'].index
    target_df = target_df.drop(drop_inx2)
    
    # 월/일별로 그룹화하여 평균 기온 반환
    mean_temp = target_df.groupby(['month', 'day']).mean()['최저기온'].values
    
    # 원본 데이터에서 최저기온 데이터 추출
    pre = df.iloc[:, -2]

    # 그룹화 하면 월 단위로 자동 정렬이 되기 때문에 인위적으로 순서 변경 필요 / 07-01은 151번째 인덱스
    mean_temp1 = mean_temp[151 : ]
    mean_temp2 = mean_temp[ : 151]

    low_temps = []

    for i in range(len(pre)):
        low_temps.append(pre.iloc[i])

    for i in range(len(mean_temp1)):
        low_temps.append(mean_temp1[i])

    for i in range(len(mean_temp2)):
        low_temps.append(mean_temp2[i])
    
    return low_temps

In [129]:
# 도봉구 최저기온 추가
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/최저기온/도봉제1동_일최저기온_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/최저기온/도봉제1동_일최저기온_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/도봉구/최저기온/도봉제1동_일최저기온_202101_202106.csv')


a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df2['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','최저기온']
b.columns = ['일','측정시간','이후시간예측','최저기온']
c.columns = ['일','측정시간','이후시간예측','최저기온']

print(a.shape, b.shape, c.shape)

temp_df = pd.concat([a, b, c])
temp_df['date'] = date_list2

low_temps = low_temp_list(temp_df)

# 구별로 합친 데이터프레임 불러오기
concated_df = pd.read_csv('./도봉구기상데이터.csv')
concated_df['최저기온'] = low_temps

concated_df.to_csv('도봉구기상데이터.csv',index=False,encoding='utf-8-sig')

concated_df


(365, 4) (366, 4) (181, 4)


,Unnamed: 0,Date,강수량,강설량,최고기온,최저기온,습도,강수형태,강수확률
0,0,2019-01-01,0.0,0.0,0.0,-9.0,40.0,0.0,0.0
1,1,2019-01-02,0.0,0.0,2.0,-7.0,35.0,0.0,0.0
2,2,2019-01-03,0.0,0.0,4.0,-7.0,50.0,0.0,0.0
3,3,2019-01-04,0.0,0.0,1.0,-8.0,70.0,0.0,20.0
4,4,2019-01-05,0.0,0.0,1.0,-9.0,30.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...
1242,1242,2022-05-27,0.0,0.0,28.0,13.5,50.0,0.0,0.0
1243,1243,2022-05-28,0.0,0.0,29.0,14.0,50.0,0.0,0.0
1244,1244,2022-05-29,0.0,0.0,24.0,16.5,55.0,0.0,0.0
1245,1245,2022-05-30,0.0,0.0,26.0,16.5,75.0,0.0,30.0


In [130]:
# 노원구 최저기온 추가
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/최저기온/공릉1동_일최저기온_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/최저기온/공릉1동_일최저기온_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/노원구/최저기온/공릉1동_일최저기온_202101_202106.csv')



a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df2['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','최저기온']
b.columns = ['일','측정시간','이후시간예측','최저기온']
c.columns = ['일','측정시간','이후시간예측','최저기온']

print(a.shape, b.shape, c.shape)

temp_df = pd.concat([a, b, c])
temp_df['date'] = date_list2

low_temps = low_temp_list(temp_df)

# 구별로 합친 데이터프레임 불러오기
concated_df = pd.read_csv('./노원구기상데이터.csv')
concated_df['최저기온'] = low_temps

concated_df.to_csv('노원구기상데이터.csv',index=False,encoding='utf-8-sig')

concated_df


(365, 4) (366, 4) (181, 4)


,Unnamed: 0,Date,강수량,강설량,최고기온,최저기온,습도,강수형태,강수확률
0,0,2019-01-01,0.0,0.0,0.0,-8.0,40.0,0.0,0.0
1,1,2019-01-02,0.0,0.0,2.0,-6.0,25.0,0.0,0.0
2,2,2019-01-03,0.0,0.0,4.0,-6.0,45.0,0.0,0.0
3,3,2019-01-04,0.0,0.0,1.0,-7.0,60.0,0.0,20.0
4,4,2019-01-05,0.0,0.0,2.0,-7.0,25.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...
1242,1242,2022-05-27,0.0,0.0,29.0,14.5,50.0,0.0,0.0
1243,1243,2022-05-28,0.0,0.0,29.0,16.0,45.0,0.0,0.0
1244,1244,2022-05-29,0.0,0.0,24.0,17.5,45.0,0.0,0.0
1245,1245,2022-05-30,0.0,0.0,27.0,17.5,70.0,0.0,30.0


In [131]:
# 은평구 최저기온 추가
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/최저기온/갈현제1동_일최저기온_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/최저기온/갈현제1동_일최저기온_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/은평구/최저기온/갈현제1동_일최저기온_202101_202106.csv')


a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df2['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','최저기온']
b.columns = ['일','측정시간','이후시간예측','최저기온']
c.columns = ['일','측정시간','이후시간예측','최저기온']

print(a.shape, b.shape, c.shape)

temp_df = pd.concat([a, b, c])
temp_df['date'] = date_list2

low_temps = low_temp_list(temp_df)

# 구별로 합친 데이터프레임 불러오기
concated_df = pd.read_csv('./은평구기상데이터.csv')
concated_df['최저기온'] = low_temps

concated_df.to_csv('은평구기상데이터.csv',index=False,encoding='utf-8-sig')

concated_df


(365, 4) (366, 4) (181, 4)


,Unnamed: 0,Date,강수량,강설량,최고기온,최저기온,습도,강수형태,강수확률
0,0,2019-01-01,0.0,0.0,-1.0,-10.0,45.0,0.0,0.0
1,1,2019-01-02,0.0,0.0,2.0,-7.0,35.0,0.0,0.0
2,2,2019-01-03,0.0,0.0,4.0,-8.0,50.0,0.0,0.0
3,3,2019-01-04,0.0,0.0,1.0,-8.0,70.0,0.0,20.0
4,4,2019-01-05,0.0,0.0,1.0,-9.0,30.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...
1242,1242,2022-05-27,0.0,0.0,29.0,13.0,55.0,0.0,0.0
1243,1243,2022-05-28,0.0,0.0,29.0,14.0,55.0,0.0,0.0
1244,1244,2022-05-29,0.0,0.0,24.0,16.0,55.0,0.0,0.0
1245,1245,2022-05-30,0.0,0.0,26.0,16.5,80.0,0.0,30.0


In [132]:
# 서대문구 최저기온 추가
df1 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/최저기온/남가좌제1동_일최저기온_201901_201912.csv')
df2 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/최저기온/남가좌제1동_일최저기온_202001_202012.csv')
df3 = pd.read_csv('C:/Users/km022/Desktop/포트홀데이터/서대문구/최저기온/남가좌제1동_일최저기온_202101_202106.csv')


a = df1[(df1['hour']==200)&(df1['forecast']==7)]
b = df2[(df2['hour']==200)&(df2['forecast']==7)]
c = df3[(df3['hour']==200)&(df2['forecast']==7)]

a.columns = ['일','측정시간','이후시간예측','최저기온']
b.columns = ['일','측정시간','이후시간예측','최저기온']
c.columns = ['일','측정시간','이후시간예측','최저기온']

print(a.shape, b.shape, c.shape)

temp_df = pd.concat([a, b, c])
temp_df['date'] = date_list2

low_temps = low_temp_list(temp_df)

# 구별로 합친 데이터프레임 불러오기
concated_df = pd.read_csv('./서대문구기상데이터.csv')
concated_df['최저기온'] = low_temps

concated_df.to_csv('서대문구기상데이터.csv',index=False,encoding='utf-8-sig')

concated_df


(365, 4) (366, 4) (181, 4)


,Unnamed: 0,Date,강수량,강설량,최고기온,최저기온,습도,강수형태,강수확률
0,0,2019-01-01,0.0,0.0,-1.0,-7.0,40.0,0.0,0.0
1,1,2019-01-02,0.0,0.0,2.0,-5.0,30.0,0.0,0.0
2,2,2019-01-03,0.0,0.0,4.0,-7.0,25.0,0.0,0.0
3,3,2019-01-04,0.0,0.0,1.0,-7.0,65.0,0.0,20.0
4,4,2019-01-05,0.0,0.0,1.0,-7.0,25.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...
1242,1242,2022-05-27,0.0,0.0,29.0,13.5,50.0,0.0,0.0
1243,1243,2022-05-28,0.0,0.0,29.0,15.5,55.0,0.0,0.0
1244,1244,2022-05-29,0.0,0.0,24.0,17.0,55.0,0.0,0.0
1245,1245,2022-05-30,0.0,0.0,26.0,17.5,75.0,0.0,30.0
